In [ ]:
import pandas as pd
import re

In [ ]:
rivals_df = pd.read_csv("rivals_data_fuzzy.csv")

rivals_data contains the firm names that need to be matched to the Siren database. It contains a 2-digit-industry code apen2, the firm name RIVAL_NAME, the year of the M&A from which the firms are a ected, and a firm id id_rival. I already took out the firms for which I got an exact match on firm name x industry.  

In [ ]:
rivals_df.head(10) #apen2

: 

In [7]:
rivals_df.shape

(2343, 5)

In [8]:
rivals_df.RIVAL_NAME

0               1001PNEUS
1               123 PNEUS
2               123 PNEUS
3                123PNEUS
4                    1664
              ...        
2338              ZANNIER
2339               ZODIAK
2340              ZOLLNER
2341       ZOO DE BEAUVAL
2342    ZOO DE LA PALMYRE
Name: RIVAL_NAME, Length: 2343, dtype: object

sirens contains all registered firms in France. It contains the firm id Siren (which is the variable I need from the matching), the firm names RIVAL_NAME, the 2-digit industry code apen2 and a random id_number id_siren.  

In [9]:
sirens_df = pd.read_csv("siren_database.csv")

In [10]:
sirens_df.head()

,siren,RIVAL_NAME,apen2,id_siren
0,785512377,NaN,67.0,1
1,438975849,\t 'ASSOCIATION DES COMBATTANTS VOLONTAIRES H...,94.0,2
2,394403679,\t ACTIONS POUR LE DEVELOPPEMENT DE LA DIGNIT...,94.0,3
3,931796742,\t AD BATIMENT,41.0,4
4,805385978,\t AILLAS GYMNASTIQUE LOISIRS,93.0,5


In [11]:
sirens_df.shape

(13508841, 4)

In [12]:
sirens_df = sirens_df.dropna(subset=['apen2']) # dropped those rows where apen2 is NA

In [13]:
sirens_df.apen2 = sirens_df.apen2.astype(int)

In [14]:
rival_name = rivals_df.loc[0].RIVAL_NAME
alternate_matches = [rival_name,
                     re.sub(r"\s+", "", rival_name),
                     re.sub(r"(\d)\s+", r"\1", rival_name),
                     re.sub(r"(\d)\s*(?=[A-Za-z])", r"\1 ", rival_name)
                     ]
alternate_matches = set(alternate_matches)
alternate_matches

{'1001 PNEUS', '1001PNEUS'}

In [43]:
sirens_df.RIVAL_NAME = sirens_df.RIVAL_NAME.astype(str)
rivals_df.RIVAL_NAME = rivals_df.RIVAL_NAME.astype(str)

In [113]:
def match_firm(rival_name, apen2):
    #print(apen2)
    rival_name = str(rival_name)
    if len(rival_name)>5:
        alternate_matches = [rival_name,
                     re.sub(r"\s+", "", rival_name),
                     re.sub(r"(\d)\s+", r"\1", rival_name),
                     re.sub(r"(\d)\s*(?=[A-Za-z])", r"\1 ", rival_name)
                     ]
        alternate_matches = set(alternate_matches)
    #print(alternate_matches)
        selected_df = sirens_df[sirens_df.apen2 == apen2]
    #print(selected_df)
    #for rival_match in alternate_matches:
        #if len(selected_df[(rival_match in selected_df.RIVAL_NAME) | (rival_match == selected_df.RIVAL_NAME)])>0:
        #        print("FOUND:",rival_match)
        pattern = "|".join(list(alternate_matches))
        df_match = selected_df[selected_df['RIVAL_NAME'].str.contains(pattern, case=False, na=False)]
        if len(df_match)>0:
            df_match["RIVAL_NAME_ORIGINAL"]=[rival_name]*len(df_match)
            df_match.to_csv("results.csv", mode='a', index=False, header=False)
        else:
            print(pattern, apen2)
    #return df_match
    

In [115]:
selected_df = sirens_df[sirens_df.apen2 == 1]
selected_df[selected_df['RIVAL_NAME'].str.contains("ACOLYANCE", case=False, na=False)]

,siren,RIVAL_NAME,apen2,id_siren


In [117]:
sirens_df[sirens_df['RIVAL_NAME'].str.contains("ACTURA", case=False, na=False)]

,siren,RIVAL_NAME,apen2,id_siren
93344,338513799,A A C O R ASSISTANCE - ASSISTANCE - AFFACTURAG...,82,93345
165196,928315175,A.R FACTURATION,82,165197
199412,899597934,AB FACTURATION,82,199413
305488,478938350,ACTURA,70,305489
305489,879603009,ACTURA,82,305490
...,...,...,...,...
12092854,791575871,SOCIETE DE FACTURATION PARAMEDICALE,82,12092855
12279946,517573960,SOURCES ALMA FACTURATION,82,12279947
12771197,839839255,TELETRANSMISSION FACTURATION POUR INFIRMIERS L...,82,12771198
13184949,840611529,VAR FACTURATION,82,13184950


In [114]:
rivals_df.apply(lambda x: match_firm(x["RIVAL_NAME"], x["apen2"]), axis =1)

123PNEUS|123 PNEUS 22
123PNEUS|123 PNEUS 45
123PNEUS|123 PNEUS 45
20MINUTES.FR|20 MINUTES.FR 63
231EAST STREET|231EASTSTREET|231 EAST STREET 56
7ACCUEIL|7 ACCUEIL 82
A2 MICILE|A2MICILE 86
ACCUEILNEGOCE|ACCUEIL NEGOCE 46
ACOLYANCE 1
ACTICALL SITEL|ACTICALLSITEL 82
ACTISEM 1
ACTURA 1
ADISERE|AD ISERE 45
ADAMA FRANCE|ADAMAFRANCE 1
ADELSCOTT 11
ADREXO 58
ADVINI 11
AEROPORTDEGENEVE|AEROPORT DE GENEVE 52
AESICA 21
AFATSELECTOUR|AFAT SELECTOUR 79
AG2 R-LA MONDIALE|AG2R-LAMONDIALE|AG2R-LA MONDIALE 65
AGIP/ENI 47
AGRIALTERNATIVE|AGRI ALTERNATIVE 1
AGRI LISA|AGRILISA 1
AGRI-APPRO 1
AGRI-DEMETER 1
AGRI-NEGOCE SA|AGRI-NEGOCESA 1
AGRIAFFAIRES.COM 63
AGRIAL 10
AGRICENTREDUMAS|AGRICENTRE DUMAS 1
AGRIFOURNITURES 1
AGRIHUB 1
AGRILEADER 1
AGRILIB 1
AGRIOBTENTIONS 1
AGRO SERVICE 2000|AGROSERVICE2000 1
AIR LIQUIDE|AIRLIQUIDE 86
AKERIS 68
AKKA TECHNOLOGIES|AKKATECHNOLOGIES 62
AKKA TECHNOLOGIES|AKKATECHNOLOGIES 71
ALJAZEERASPORTFRANCE(BEINSPORT)|AL JAZEERA SPORT FRANCE (BEIN SPORT) 60
ALAZARDETROUX|ALAZARD 

C:\Users\sukan\AppData\Local\Temp\ipykernel_47832\2427231267.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_match = selected_df[selected_df['RIVAL_NAME'].str.contains(pattern, case=False, na=False)]


ALCODIS 10
ALICAL 10
ALIFEL SAS|ALIFELSAS 1
ALITMATE 46
ALLIANCEINDUSTRIE|ALLIANCE INDUSTRIE 45
ALLIANCE PASTORALE|ALLIANCEPASTORALE 1
ALLIANCE/CEDEM 10
ALLIANZGENERALI|ALLIANZ GENERALI 66
ALLO BEBE|ALLOBEBE 47
ALLOPNEUS 22
ALPLA WERKE|ALPLAWERKE 22
ALSTOM 42
ALTERA-COGEDIM 41
ALTERGAZ 35
ALTERNATIVEMOBILE|ALTERNATIVE MOBILE 61
ALTICE 47
ALYZIASURETE|ALYZIA SURETE 80
AMORA/MAILLE|AMORA / MAILLE 10


KeyboardInterrupt: 

In [104]:
results_df = pd.read_excel("Results.xlsx")

In [106]:
missing_matches = set(rivals_df.RIVAL_NAME.to_list()).difference(set(results_df.RIVAL_NAME_ORIGINAL.to_list()))

In [109]:
missing_matches

{'GROUPE CARREFOUR',
 'NETFLIX',
 'KRAMP',
 'INFUN',
 'CEFLA',
 'CLINIQUE MEDIPOLE GARONNE',
 'PAUWELS',
 'IPSANTE DOMICILE',
 'COPRODEX',
 'THIMMONIER',
 'MINDRAY',
 'CHAMPAGNES CEREALES',
 'RAGT',
 'J’AIME LIRE',
 'AGRIAFFAIRES.COM',
 'ASSISTANCE PUBLIQUE – HOPITAUX DE PARIS',
 'MEIJER FRANCE SARL',
 'ANTARGAZ',
 'KOCH',
 'SOCIETES D’INVESTISSEMENT IMMOBILIER COTEES',
 'NESTLE HOMECARE',
 'ELLE',
 'CARREFOUR LOCATION',
 'EUROMASTER (MICHELIN)',
 'HEINEKEN',
 'MENORQUINA',
 'AMORA / MAILLE',
 'MATFOR',
 'ETABLISSEMENTS THIELIN',
 'GROUPE MALAKOFF-MEDERIC',
 'COCORETTE',
 'LISTEL',
 'GROUPE VIVIER BEGANTON',
 'NUTRIXO (FRANCINE)',
 'KRONOFRANCE',
 'SOPRA',
 'COOPERATIVE DE JUNIVILLE',
 'JOA',
 'BRICOLECLERC',
 'QUALISOL',
 'XEROX',
 'LOUIS DELHAIZE (CORA)',
 'ISP',
 'COMPAGNIES D’ASSURANCE',
 'LAYER',
 'CYRILLUS-VERBAUDET',
 'WARTSILLA',
 'HUF',
 'UNION DE COOPERATIVES FEDER',
 'FRUIDOR',
 "L'EQUIPE TV",
 'MCCORMICK (VAHINE)',
 'NRJ GLOBAL',
 'SPIE SUD EST',
 'CRISTALCO',
 'COFELY ENDE

In [108]:
len(rivals_df) - len(missing_matches)

779

In [38]:
all_firms = sirens_df.RIVAL_NAME.to_list()
all_firms = list(map(str, all_firms))

In [110]:
for firm in all_firms:
    if re.search("HEINEKEN", firm):
        print(firm)

BRASSERIES HEINEKEN SA
BRASSERIES HEINEKEN SNC
CE HEINEKEN
COMITE ETABLISSEMENT BRASSERIES HEINEKEN
CSE HEINEKEN SIEGE & FORCE DE VENTE
HEINEKEN ANTILLES-GUYANE (FWI)
HEINEKEN BROUWERIJEN B V
HEINEKEN BROVWERIJE BV
HEINEKEN ENTREPRISE
HEINEKEN FRANCE
HEINEKEN FRANCE
HEINEKEN GLOBAL PROCUREMENT B.V.
HEINEKEN GROUP BV
HEINEKEN INTERNATIONAAL BEHEER B V
HEINEKEN INTERNATIONAL B.V.
HEINEKEN NV
HEINEKEN ST MARTIN
HEINEKEN TECHNISCH BEHEER
ROEMINCK INSURANCE NV AND HEINEKEN NV
SENIOR CLUB HEINEKEN
SOCIETE NOUVELLE PUB HEINEKEN


In [57]:
"20 MINUTES" in "20 MINUTES FRANCE"

True

In [58]:
sirens_df[sirens_df.RIVAL_NAME=="20 MINUTES FRANCE"]

,siren,RIVAL_NAME,apen2,id_siren
24177,438049843,20 MINUTES FRANCE,58,24178


In [59]:
rivals_df[rivals_df.RIVAL_NAME=="20 MINUTES"]

,apen2,local,RIVAL_NAME,year_event,id_rival
6,58,0,20 MINUTES,2015,9
